<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: spec2, flat_field step

<span style="color:red"> **Instruments Affected**</span>: NIRSpec 

Tested on CV3 data

### Table of Contents
<div style="text-align: left"> 

<br> [Imports](#imports_ID) <br> [Introduction](#intro_ID) <br> [Testing Data Set](#data_ID) <br> [Run the JWST pipeline and flat_field validation tests](#pipeline_ID): [FS Full-Frame test](#FULLFRAME), [FS ALLSLITS test](#ALLSLITS), [MOS test](#MOS), [IFU test](#IFU) <br> [About This Notebook](#about_ID)<br> [Results](#results) <br>

</div>

<a id="imports_ID"></a>
# Imports
The library imports relevant to this notebook are aready taken care of by importing PTT.

* astropy.io for opening fits files
* jwst.module.PipelineStep is the pipeline step being tested
* matplotlib.pyplot.plt to generate plot

NOTE: This notebook assumes that the pipeline version to be tested is already installed and its environment is activated.

To be able to run this notebook you need to install nptt. 

If all goes well you will be able to import PTT.

[Top of Page](#title_ID)

In [1]:
import os
import subprocess
from astropy.io import fits
from glob import glob

from jwst.pipeline.calwebb_detector1 import Detector1Pipeline
from jwst.assign_wcs.assign_wcs_step import AssignWcsStep
from jwst.msaflagopen.msaflagopen_step import MSAFlagOpenStep
from jwst.extract_2d.extract_2d_step import Extract2dStep
from jwst.srctype.srctype_step import SourceTypeStep
from jwst.flatfield.flat_field_step import FlatFieldStep

# The latest version of NPTT is installed in the requirements text file at:
# /jwst_validation_notebooks/environment.yml

# import NPTT
import nirspec_pipe_testing_tool as nptt

# To get data from Artifactory
from ci_watson.artifactory_helpers import get_bigdata



In [2]:
# Make sure that the version used is the right one

# to get the latest version from 'master' run:
# subprocess.run(['pip', 'install', 'git+https://github.com/spacetelescope/nirspec_pipe_testing_tool@master'])

latest_nptt_stable_version = '1.1.4'
nptt_version = nptt.__version__

if nptt_version != latest_nptt_stable_version:
    print("* WARNING - The code might break because the version used is not the latest stable version:")
    print("            Latest stable version is ", latest_nptt_stable_version)
    print("            Using NPTT version ", nptt_version)
else:
    print("NPTT version is correct.")

NPTT version is correct.


<a id="intro_ID"></a>
# Test Description

The test is a direct comparison of the result of our implementation of the flat field step algorithm versus the pipeline's implementation, i.e.: 
              difference =  absolute( Flat_nirspec_implementation - Flat_pipeline)

We expect the absolute difference to be of the order of 1x10^-6. We set this threshold by assuming that the difference should yield computer precision 1x10^-7 numbers. We then relaxed one order of magnitude due to interpolation differences in the algorithms.

For the test to be considered PASSED, every single slit (for FS data), slitlet (for MOS data) or slice (for IFU data) in the input file has to pass. If there is any failure, the whole test will be considered as FAILED. 

The code for this test for Fixed Slits (FS) can be obtained from: https://github.com/spacetelescope/nirspec_pipe_testing_tool/blob/master/nirspec_pipe_testing_tool/calwebb_spec2_pytests/auxiliary_code/flattest_fs.py. For Multi Object Spectroscopy (MOS), the code is in the same repository but is named ```flattest_mos.py```, and for Integral Field Unit (IFU) data, the test is named ```flattest_ifu.py```.

The input file is defined in the variable ```input_file``` (see section [Testing Data Set and Variable Setup](#data_ID)).

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/flatfield/main.html

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/flatfield


### Test Results

If the test **PASSED** this means that all slits, slitlets, or slices individually passed the test. However, if ony one individual slit (for FS data), slitlet (for MOS data) or slice (for IFU data) test failed, the whole test will be reported as **FAILED**.


### Calibration WG Requested Algorithm: 

A short description and link to the page: https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Spectral+Flat+Field+Correction


### Defining Term
Acronymns used un this notebook:

pipeline: calibration pipeline

spec2: spectroscopic calibration pipeline level 2b

PTT: NIRSpec pipeline testing tool (https://github.com/spacetelescope/nirspec_pipe_testing_tool)


[Top of Page](#title_ID)

<a id="pipeline_ID"></a>
# Run the JWST pipeline and assign_wcs validation tests

The pipeline can be run from the command line in two variants: full or per step.

Tu run the spec2 pipeline in full use the command: 

$ strun jwst.pipeline.Spec2Pipeline jwtest_rate.fits

Tu only run the flat_field step, use the command:

$ strun jwst.flat_field.FlatFieldStep jwtest_extract_2d.fits

These options are also callable from a script with the testing environment active. The Python call for running the pipeline in full or by step are:

$\gt$ from jwst.pipeline.calwebb_spec2 import Spec2Pipeline

$\gt$ Spec2Pipeline.call(jwtest_rate.fits)
 
or
 
$\gt$ from jwst.flat_field.flat_field_step import FlatFieldStep
 
$\gt$ FlatFieldStep.call(jwtest_extract_2d.fits)

For the imaging pipeline the call would be as follows:

$\gt$ from jwst.pipeline.calwebb_image2 import Image2Pipeline

$\gt$ Image2Pipeline.call(jwtest_rate.fits)

NPTT can run the spec2 pipeline either in full or per step, as well as the imaging pipeline in full. In this notebook we will use NPTT to run the pipeline and the validation tests. To run NPTT, follow the directions in the corresponding repo page.

[Top of Page](#title_ID)

-> For each mode, the following variables will need to be set:
- output_directory = string, path where you want intermediary files and plots to be saved in, if writefile=True
- input_file = string or object, this is the output file from the previous step, e.g. jwtest1_NRS1_extract2d.fits
- dflat_path = string, path of where the D-flat reference fits files
- sflat_path = string, path of where the S-flat reference fits files
- fflat_path = string, path of where the F-flat reference fits files
- writefile = boolean, if True writes the fits files of the calculated flat, and the difference jpeg images
- save_figs = boolean, whether to save plots or not
- show_figs = boolean, whether to show plots or not 
- threshold_diff = float, threshold difference between pipeline output and ESA file


<a id="data_ID"></a>
# Testing Data Set

All testing data is from the CV3 campaign. We chose these files because this is our most complete data set, i.e. all modes and filter-grating combinations.

Data used was for testing:
- FS_PRISM_CLEAR
- FS_FULLFRAME_G395H_F290LP
- FS_ALLSLITS_G140H_F100LP 
- MOS_G140M_LINE1 
- MOS_PRISM_CLEAR
- IFU_G395H_F290LP 


[Top of Page](#title_ID)

In [7]:
testing_data = {'fs_prism_clear':{
                                  'uncal_file_nrs1': 'fs_prism_nrs1_uncal.fits',
                                  'uncal_file_nrs2': 'fs_prism_nrs2_uncal.fits',
                                  'sflat_nrs1': 'nirspec_FS_sflat_PRISM_OPAQUE_FLAT5_nrs1_f_01.01.fits',
                                  'sflat_nrs2': 'nirspec_FS_sflat_PRISM_OPAQUE_FLAT5_nrs2_f_01.01.fits',
                                  'fflat': 'nirspec_FS_fflat_CLEAR_01.01.fits',
                                  'msa_shutter_config': None },
                
                'fs_fullframe_g395h_f290lp':{
                                  'uncal_file_nrs1': 'fs_fullframe_g35h_f290lp_nrs1_uncal.fits',
                                  'uncal_file_nrs2': 'fs_fullframe_g35h_f290lp_nrs2_uncal.fits',
                                  'sflat_nrs1': 'nirspec_FS_sflat_G395H_OPAQUE_FLAT3_nrs1_f_01.01.fits',
                                  'sflat_nrs2': 'nirspec_FS_sflat_G395H_OPAQUE_FLAT3_nrs2_f_01.01.fits',  
                                  'fflat': 'nirspec_FS_fflat_F290LP_01.01.fits',
                                  'msa_shutter_config': None },
                
                'fs_allslits_g140h_f100lp':{
                                  'uncal_file_nrs1': 'fs_allslits_g140h_f100lp_nrs1_uncal.fits',
                                  'uncal_file_nrs2': 'fs_allslits_g140h_f100lp_nrs2_uncal.fits',
                                  'sflat_nrs1': 'nirspec_FS_sflat_G140H_OPAQUE_FLAT1_nrs1_f_01.01.fits',
                                  'sflat_nrs2': 'nirspec_FS_sflat_G140H_OPAQUE_FLAT1_nrs2_f_01.01.fits',
                                  'fflat': 'nirspec_FS_fflat_F100LP_01.01.fits',
                                  'msa_shutter_config': None },
                
                'mos_prism_clear':{
                                  'uncal_file_nrs1': 'mos_prism_nrs1_uncal.fits',
                                  'uncal_file_nrs2': 'mos_prism_nrs2_uncal.fits',
                                  'sflat_nrs1': 'nirspec_MOS_sflat_PRISM_OPAQUE_FLAT5_nrs1_f_01.01.fits',
                                  'sflat_nrs2': 'nirspec_MOS_sflat_PRISM_OPAQUE_FLAT5_nrs2_f_01.01.fits',
                                  'fflat': 'nirspec_MOS_fflat_CLEAR_01.01.fits',
                                  'msa_shutter_config': 'V0030006000104_msa.fits' },
                
                'mos_g140m_f100lp':{
                                  'uncal_file_nrs1': 'mos_g140m_line1_NRS1_uncal.fits',
                                  'uncal_file_nrs2': 'mos_g140m_line1_NRS2_uncal.fits',  
                                  'sflat_nrs1': 'nirspec_MOS_sflat_G140M_OPAQUE_FLAT1_nrs1_f_01.01.fits',
                                  'sflat_nrs2': 'nirspec_MOS_sflat_G140M_OPAQUE_FLAT1_nrs2_f_01.01.fits',
                                  'fflat': 'nirspec_MOS_fflat_F100LP_01.01.fits',
                                  'msa_shutter_config': 'V8460001000101_msa.fits' },
                
                
                'ifu_g395h_f290lp':{
                                  'uncal_file_nrs1': 'ifu_g395h_f290lp_nrs1_uncal.fits',
                                  'uncal_file_nrs2': 'ifu_g395h_f290lp_nrs2_uncal.fits',
                                  'sflat_nrs1': 'nirspec_IFU_sflat_G395H_OPAQUE_FLAT3_nrs1_f_01.01.fits',
                                  'sflat_nrs2': 'nirspec_IFU_sflat_G395H_OPAQUE_FLAT3_nrs2_f_01.01.fits',
                                  'fflat': 'nirspec_IFU_fflat_F290LP_01.01.fits',
                                  'msa_shutter_config': None }

               }


2021-01-20 16:18:56,668 - stpipe - WARNING - /Users/pena/anaconda3/envs/pipedev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



In [8]:
# define function to pull data from Artifactory
def get_artifactory_file(data_set_dict, detector):
    """This function creates a list with all the files needed per detector to run the test.
    Args:
        data_set_dict: dictionary, contains inputs for a specific mode and configuration
        detector: string, either nrs1 or nrs2
    Returns:
        data: list, contains all files needed to run test
    """
    files2obtain = ['uncal_file_nrs1', 'sflat_nrs1', 'fflat', 'msa_shutter_config']
    data = []
    for file in files2obtain:
        data_file = None
        try: 
            if '_nrs' in file and '2' in detector:
                file = file.replace('_nrs1', '_nrs2')

            data_file = get_bigdata('jwst_validation_notebooks',
                                         'validation_data',
                                         'nirspec_data', 
                                         data_set_dict[file])
        except TypeError:
            data.append(None)
            continue

        data.append(data_file)

    return data
    

In [9]:
# set the D-flat path (used for all test data)
dflat = get_bigdata('jwst_validation_notebooks',
                         'validation_data',
                         'nirspec_data', 
                         'nirspec_dflat_nrs1_f_01.03.fits')

# accepted threshold difference with respect to benchmark files
threshold_diff = 9.999e-5

# set NPTT switches for this test
writefile = False
save_figs = False
show_figs = True


# Get the data
results_dict = {}
detectors = ['nrs1', 'nrs2']
for mode_config, data_set_dict in testing_data.items():
    for det in detectors:
        print('Testing files for detector: ', det)
        data = get_artifactory_file(data_set_dict, det)
        uncal_file, sflat, fflat, msa_shutter_config = data
        print('Working with uncal_file: ', uncal_file)
        uncal_basename = os.path.basename(uncal_file)
        
        # Make sure there is a local copy of the MSA shutter configuration file
        if msa_shutter_config is not None:
            subprocess.run(['cp', msa_shutter_config , '.'])
        
        # Run the stage 1 pipeline 
        rate_object = Detector1Pipeline.call(uncal_file)
        # Make sure the MSA shutter configuration file is set up correctly
        if msa_shutter_config is not None:
            msa_metadata = rate_object.meta.instrument.msa_metadata_file
            print(msa_metadata)
            if msa_metadata is None or msa_metadata == 'N/A':
                rate_object.meta.instrument.msa_metadata_file = msa_shutter_config

        # Run the stage 2 pipeline steps
        pipe_object = AssignWcsStep.call(rate_object)
        if 'ifu' in uncal_basename.lower() or 'mos' in uncal_basename.lower():
            pipe_object = MSAFlagOpenStep.call(pipe_object)
        if 'ifu' not in uncal_basename.lower():
            pipe_object = Extract2dStep.call(pipe_object)
        pipe_object = SourceTypeStep.call(pipe_object)
        flat_field_object = FlatFieldStep.call(pipe_object, save_interpolated_flat=True)

        # Run the validation test
        %matplotlib inline
        interpolated_flat = os.path.basename(uncal_file).replace('uncal', 'interpolatedflat')
        if 'fs' in uncal_file.lower():
            print('Running flat field test for FS...')
            result, result_msg, log_msgs = nptt.calwebb_spec2_pytests.auxiliary_code.flattest_fs.flattest(
                                                                            flat_field_object,
                                                                            dflat_path=dflat,
                                                                            sflat_path=sflat, 
                                                                            fflat_path=fflat,
                                                                            writefile=writefile,
                                                                            show_figs=show_figs, 
                                                                            save_figs=save_figs,
                                                                            interpolated_flat=interpolated_flat,
                                                                            threshold_diff=threshold_diff,
                                                                            debug=False)
        if 'mos' in uncal_file.lower():
            print('Running flat field test for MOS...')
            result, result_msg, log_msgs = nptt.calwebb_spec2_pytests.auxiliary_code.flattest_mos.flattest(
                                                                            flat_field_object,
                                                                            dflat_path=dflat,
                                                                            sflat_path=sflat, 
                                                                            fflat_path=fflat,
                                                                            msa_shutter_conf=msa_shutter_config,     
                                                                            writefile=writefile,
                                                                            show_figs=show_figs, 
                                                                            save_figs=save_figs,
                                                                            interpolated_flat=interpolated_flat,
                                                                            threshold_diff=threshold_diff,
                                                                            debug=False)
        if 'ifu' in uncal_file.lower():
            print('Running flat field test for IFU...')
            result, result_msg, log_msgs = nptt.calwebb_spec2_pytests.auxiliary_code.flattest_ifu.flattest(
                                                                            flat_field_object,
                                                                            dflat_path=dflat,
                                                                            sflat_path=sflat, 
                                                                            fflat_path=fflat,
                                                                            writefile=writefile,
                                                                            mk_all_slices_plt=False, 
                                                                            show_figs=show_figs, 
                                                                            save_figs=save_figs,
                                                                            interpolated_flat=interpolated_flat,
                                                                            threshold_diff=threshold_diff,
                                                                            debug=False)

        # Remove fits files from the repo directory
        local_fits_files = glob('./*.fits')
        for fits_file in local_fits_files:
            if 'dflat' not in fits_file:
                try:
                    subprocess.run(['rm', fits_file])
                except FileNotFound:
                    print('Fits file does not exist in current working directory: ', fits_file)

        # Did the test passed 
        print("Did flat_field validation test passed? ", result_msg, "\n\n")
        rd = {mode_config: result_msg}
        results_dict.update(rd)

# Remove the dflat fits file local copy
try:
    subprocess.run(['rm', os.path.basename(dflat)])
except FileNotFound:
    print('Fits file does not exist in current working directory: ', fits_file)




Testing files for detector:  nrs1
Working with uncal_file:  /Users/pena/Documents/NIRSpec_projects/pipeline_testing_notebooks/jwst_validation_notebooks/jwst_validation_notebooks/flat_field/jwst_flat_field_nirspec_test/mos_prism_nrs1_uncal.fits


2021-01-20 16:26:17,515 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-01-20 16:26:17,516 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-01-20 16:26:17,518 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-01-20 16:26:17,520 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-01-20 16:26:17,522 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-01-20 16:26:17,523 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2021-01-20 16:26:17,526 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2021-01-20 16:26:17,528 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2021-01-20 16:26:17,530 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2021-01-20 16:26:17,533 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2021-01-20 1

2021-01-20 16:26:17,994 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSAT reference file is 'N/A'.
2021-01-20 16:26:17,994 - stpipe.Detector1Pipeline - INFO - Prefetch for READNOISE reference file is '/Users/pena/crds_cache/references/jwst/nirspec/jwst_nirspec_readnoise_0018.fits'.
2021-01-20 16:26:17,995 - stpipe.Detector1Pipeline - INFO - Prefetch for REFPIX reference file is '/Users/pena/crds_cache/references/jwst/nirspec/jwst_nirspec_refpix_0022.fits'.
2021-01-20 16:26:17,996 - stpipe.Detector1Pipeline - INFO - Prefetch for RESET reference file is 'N/A'.
2021-01-20 16:26:17,996 - stpipe.Detector1Pipeline - INFO - Prefetch for RSCD reference file is 'N/A'.
2021-01-20 16:26:17,997 - stpipe.Detector1Pipeline - INFO - Prefetch for SATURATION reference file is '/Users/pena/crds_cache/references/jwst/nirspec/jwst_nirspec_saturation_0020.fits'.
2021-01-20 16:26:17,998 - stpipe.Detector1Pipeline - INFO - Prefetch for SUPERBIAS reference file is '/Users/pena/crds_cache/references/jwst

2021-01-20 16:26:20,480 - stpipe.Detector1Pipeline.superbias - INFO - Step superbias done
2021-01-20 16:26:20,584 - stpipe.Detector1Pipeline.refpix - INFO - Step refpix running with args (<RampModel(1, 4, 3200, 2048) from mos_prism_nrs1_uncal.fits>,).
2021-01-20 16:26:20,585 - stpipe.Detector1Pipeline.refpix - INFO - Step refpix parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/pena/Documents/NIRSpec_projects/pipeline_testing_notebooks/jwst_validation_notebooks/jwst_validation_notebooks/flat_field/jwst_flat_field_nirspec_test', 'odd_even_columns': True, 'use_side_ref_pixels': True, 'side_smoothing_length': 11, 'side_gain': 1.0, 'odd_even_rows': True}
2021-01-20 16:26:20,598 - stpipe.Detector1Pipeline.refpix - INFO - Using refpix reference file: /Users/pena/crds_cache/

2021-01-20 16:26:55,894 - stpipe.Detector1Pipeline - INFO - ... ending calwebb_detector1
2021-01-20 16:26:55,895 - stpipe.Detector1Pipeline - INFO - Step Detector1Pipeline done
2021-01-20 16:26:55,909 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.
2021-01-20 16:26:56,009 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args (<ImageModel(2048, 2048) from mos_prism_nrs1_uncal.fits>,).
2021-01-20 16:26:56,011 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.25, 'sip_degree': None, 'sip_max_inv_pix_error': 0.25, 'sip_inv_degree': None, 'sip_npoints': 32, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2021-01-20 16:26:56,054 - stpipe.AssignWcsStep - INFO - 

N/A


2021-01-20 16:26:56,550 - stpipe.AssignWcsStep - INFO - Slits projected on detector NRS1: [3, 4]
2021-01-20 16:26:56,550 - stpipe.AssignWcsStep - INFO - Computing WCS for 2 open slitlets
2021-01-20 16:26:56,574 - stpipe.AssignWcsStep - INFO - gwa_ytilt is 0.0332140289247036 deg
2021-01-20 16:26:56,574 - stpipe.AssignWcsStep - INFO - gwa_xtilt is 0.3367482721805573 deg
2021-01-20 16:26:56,575 - stpipe.AssignWcsStep - INFO - theta_y correction: -1.0907214098475986e-05 deg
2021-01-20 16:26:56,577 - stpipe.AssignWcsStep - INFO - theta_x correction: 0.0 deg
2021-01-20 16:26:56,600 - stpipe.AssignWcsStep - INFO - SPORDER= 0, wrange=[6e-07, 5.3e-06]
2021-01-20 16:26:56,783 - stpipe.AssignWcsStep - INFO - There are 0 open slits in quadrant 1
2021-01-20 16:26:56,784 - stpipe.AssignWcsStep - INFO - There are 0 open slits in quadrant 2
2021-01-20 16:26:56,784 - stpipe.AssignWcsStep - INFO - There are 1 open slits in quadrant 3
2021-01-20 16:26:56,827 - stpipe.AssignWcsStep - INFO - There are 1 op

2021-01-20 16:27:18,072 - stpipe.FlatFieldStep - INFO - Working on slit 3
2021-01-20 16:27:18,728 - stpipe.FlatFieldStep - INFO - Working on slit 4
2021-01-20 16:27:19,836 - stpipe.FlatFieldStep - INFO - Saved model in mos_prism_nrs1_interpolatedflat.fits
2021-01-20 16:27:19,836 - stpipe.FlatFieldStep - INFO - Interpolated flat written to "mos_prism_nrs1_interpolatedflat.fits".
2021-01-20 16:27:19,838 - stpipe.FlatFieldStep - INFO - Step FlatFieldStep done


Running flat field test for MOS...


NameError: name 'msa_shutter_conf' is not defined

In [ ]:
# Quickly see if the test passed 

print('These are the final results of the tests: ')
for key, val in results_dict.items():
    print(key, val)
    

<a id="about_ID"></a>
## About this Notebook
**Author:** Maria A. Pena-Guerrero, Staff Scientist II - Systems Science Support, NIRSpec
<br>**Updated On:** Jan/20/2021

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 